In [1]:
import sys

sys.path.append("../")

import MetaTrader5 as mt5
import pytz
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

from datetime import datetime, timedelta
from terminal.Extensions.MT5 import MT5
from terminal.Extensions.Range import Range

In [2]:
MT5.initialize()

In [3]:
slippage = 5
brick_size = 1.5

from_date = datetime(2024, 6, 10, tzinfo=pytz.utc)
to_date = datetime(2024, 6, 11, tzinfo=pytz.utc)

trades_list = None

while from_date < to_date:
    trades = mt5.copy_ticks_range("WIN$", from_date, from_date.replace(hour=23), mt5.COPY_TICKS_TRADE)
    if mt5.last_error()[0] == mt5.RES_S_OK:
        if trades_list is None: 
            trades_list = trades
        else: 
            trades_list = np.append(trades, range, axis=0)
    from_date += timedelta(days=1)
    
del trades

In [4]:
len(trades_list)

4334222

In [4]:
all_trades = MT5.create_ticks_dataframe(trades_list)
trades_range = Range(
    brick_size, all_trades.index, all_trades["last"], all_trades["volume"]
)
ohlc = pd.DataFrame(trades_range.bricks)
ohlc.index = ohlc["time"]
ohlc.drop(columns=["time"], inplace=True)

In [ ]:
fig = go.Figure(
    data=go.Ohlc(
        open=ohlc["open"],
        high=ohlc["high"],
        low=ohlc["low"],
        close=ohlc["close"],
    )
)

fig.update_layout(
    autosize=True,
    width=1500,
    height=800,
)